# Задание 2. SQL

In [7]:
import pandahouse as ph

In [8]:
connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database': 'default',
                      'user': 'student', 
                      'password': 'dpo_python_2020'}

In [9]:
q = '''
    SELECT 
    uniq(st_id) AS successful_students
FROM 
    (
    SELECT 
        st_id, 
        toStartOfMonth(timest) as current_month,
        COUNT(correct) as successful_peas
    FROM default.peas
    WHERE correct = 1
    GROUP BY st_id, current_month
    HAVING successful_peas >= 20
    ORDER BY successful_peas DESC
        )
    '''
successful_students = ph.read_clickhouse(query=q, connection=connection_default)

In [10]:
successful_students.set_index(['successful_students'])

""
successful_students
136


In [11]:
q = '''
   WITH 
    all_ids AS
        (SELECT
            test_grp,
            uniq(st_id) AS clients
        FROM
            default.studs
        GROUP BY test_grp),
        
    total_revenue AS
        (SELECT
            s.test_grp AS test_grp,
            SUM(f.money) AS total_revenue
        FROM
            default.studs AS s
        LEFT JOIN
            default.final_project_check AS f ON s.st_id = f.st_id
        GROUP BY test_grp),
        
    active_ids AS
    (SELECT
        test_grp,
        COUNT(st_id) AS active_id
    FROM
        (SELECT
            p.st_id AS st_id,
            s.test_grp AS test_grp,
            SUM(p.correct) AS correct
        FROM
            default.peas AS p 
        LEFT JOIN default.studs AS s ON p.st_id = s.st_id
        GROUP BY test_grp, st_id
        HAVING correct > 10)
    GROUP BY test_grp),
        
    active_revenue AS
    (SELECT
        test_grp,
        SUM(money) AS active_revenue
    FROM
        (SELECT
            x.test_grp AS test_grp,
            x.st_id,
            f.money AS money
        FROM
            (SELECT
                p.st_id AS st_id,
                s.test_grp AS test_grp,
                SUM(p.correct) AS correct
            FROM
                default.peas AS p 
            LEFT JOIN default.studs AS s ON p.st_id = s.st_id
            GROUP BY test_grp, st_id
            HAVING correct > 10) AS x
        LEFT JOIN default.final_project_check AS f ON x.st_id = f.st_id
        WHERE money > 0)
    GROUP BY test_grp),
        
    total_purchases AS
        (SELECT
            s.test_grp AS test_grp,
            uniq(s.st_id) as total_purchases
        FROM
            (SELECT
                st_id,
                test_grp
            FROM default.studs 
            GROUP BY st_id, test_grp) AS s
        LEFT JOIN default.final_project_check AS f ON s.st_id = f.st_id
        WHERE money > 0
        GROUP BY test_grp),

    active_purchases AS
        (SELECT
            test_grp,
            uniq(st_id) AS active_purchases
        FROM
            (SELECT
                s.st_id AS st_id,
                s.test_grp AS test_grp,
                SUM(p.correct) AS correct
            FROM 
                default.studs AS s
            LEFT JOIN
                default.peas AS p ON s.st_id = p.st_id
            GROUP BY st_id, test_grp
            HAVING correct > 10) AS x
        LEFT JOIN
            default.final_project_check AS f ON x.st_id = f.st_id
        WHERE f.money > 0
        GROUP BY test_grp),

    math_ids AS
        (SELECT
            test_grp,
            uniq(st_id) as active_math
        FROM
            (SELECT
                p.st_id AS st_id,
                s.test_grp AS test_grp,
                SUM(p.correct) AS correct
            FROM
                default.studs AS s
            LEFT JOIN default.peas AS p ON s.st_id = p.st_id
            WHERE p.subject = 'Math'
            GROUP BY st_id, test_grp
            HAVING correct > 1)
        GROUP BY test_grp),

    math_purchases AS 
    (SELECT
        test_grp,
        uniq(st_id) AS math_purchases
    FROM
        (SELECT
            p.st_id AS st_id,
            s.test_grp AS test_grp,
            SUM(p.correct) AS correct
        FROM
            default.studs AS s
        LEFT JOIN default.peas AS p ON s.st_id = p.st_id
        WHERE p.subject = 'Math'
        GROUP BY st_id, test_grp
        HAVING correct > 1) AS x
    LEFT JOIN default.final_project_check AS f ON x.st_id = f.st_id
    WHERE f.money > 0 and f.subject = 'Math'
    GROUP BY test_grp)
        
    SELECT
        all_ids.test_grp AS test_grp,
        ROUND(total_revenue.total_revenue / all_ids.clients, 2) AS ARPU,
        ROUND(active_revenue.active_revenue / active_ids.active_id, 2) AS ARPAU,
        ROUND(total_purchases.total_purchases / all_ids.clients * 100, 2) AS CR,
        ROUND(active_purchases.active_purchases / active_ids.active_id * 100, 2) AS CR_active,
        ROUND(math_purchases.math_purchases /math_ids.active_math * 100, 2) AS CR_math
    FROM
        all_ids
    JOIN total_revenue ON all_ids.test_grp = total_revenue.test_grp
    JOIN active_ids ON all_ids.test_grp = active_ids.test_grp
    JOIN active_revenue ON all_ids.test_grp = active_revenue.test_grp
    JOIN total_purchases ON all_ids.test_grp = total_purchases.test_grp
    JOIN active_purchases ON all_ids.test_grp = active_purchases.test_grp
    JOIN math_purchases ON all_ids.test_grp = math_purchases.test_grp
    JOIN math_ids ON all_ids.test_grp = math_ids.test_grp
    '''
platform_metrics = ph.read_clickhouse(query=q, connection=connection_default)

In [12]:
platform_metrics

,test_grp,ARPU,ARPAU,CR,CR_active,CR_math
0,control,4540.98,10393.70,4.92,11.02,6.12
1,pilot,11508.47,29739.58,10.85,26.04,9.52
